In [4]:
# === Step 1: Import libraries ===
import numpy as np
import pandas as pd

# === Step 2: Generate synthetic Palantir stock data ===
np.random.seed(42)
n = 252  # ~1 trading year
dates = pd.bdate_range("2024-01-01", periods=n)

# Geometric Brownian Motion for close price
S0, mu, sigma, dt = 25, 0.15, 0.3, 1/252
price = [S0]
for _ in range(1, n):
    shock = np.random.normal((mu - 0.5*sigma**2)*dt, sigma*np.sqrt(dt))
    price.append(price[-1] * np.exp(shock))

df = pd.DataFrame(index=dates)
df["open"] = np.array(price) * (1 + np.random.normal(0, 0.002, size=n))
df["close"] = np.array(price)
df["high"] = df[["open", "close"]].max(axis=1) * (1 + np.random.uniform(0.001, 0.015, size=n))
df["low"] = df[["open", "close"]].min(axis=1) * (1 - np.random.uniform(0.001, 0.015, size=n))
df["volume"] = np.random.randint(100000, 500000, size=n)

# Inject missingness
df.loc[np.random.choice(df.index, 5, replace=False), "close"] = np.nan
df.loc[np.random.choice(df.index, 5, replace=False), "volume"] = np.nan

# Inject outliers
df.loc[np.random.choice(df.index, 2, replace=False), "high"] *= 1.5
df.loc[np.random.choice(df.index, 2, replace=False), "volume"] *= 5

# === Step 3: Create engineered features ===
# Feature 1: Daily Return
df['daily_return'] = df['close'].pct_change()

# Feature 2: 7-Day Rolling Volatility
df['rolling_volatility'] = df['daily_return'].rolling(window=7).std()

# Feature 3: Price-to-Volume Ratio
df['price_volume_ratio'] = df['close'] / df['volume']

df['rolling_volatility'] = df['rolling_volatility'].fillna(0)

df.head(10)


C:\Users\kvssr\AppData\Local\Temp\ipykernel_38144\3995588913.py:34: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['daily_return'] = df['close'].pct_change()


,open,close,high,low,volume,daily_return,rolling_volatility,price_volume_ratio
2024-01-01,25.045893,25.000000,25.106628,24.871937,264134.0,NaN,0.000000,0.000095
2024-01-02,25.353451,25.246297,25.433061,24.885822,175713.0,0.009852,0.000000,0.000144
2024-01-03,25.242928,25.190910,25.355092,24.896416,318335.0,-0.002194,0.000000,0.000079
2024-01-04,25.434250,25.511774,25.594675,25.358924,175418.0,0.012737,0.000000,0.000145
2024-01-05,26.242240,26.267680,26.362557,25.896931,228232.0,0.029630,0.000000,0.000115
2024-01-08,26.228890,26.162599,26.359807,25.957902,263038.0,-0.004000,0.000000,0.000099
2024-01-09,26.021065,26.057946,26.147253,25.669163,268383.0,-0.004000,0.000000,0.000097
2024-01-10,26.882378,26.858537,27.246761,26.530918,393719.0,0.030724,0.015066,0.000068
2024-01-11,27.304503,27.262266,27.362478,27.072712,425996.0,0.015032,0.015162,0.000064
2024-01-12,26.982605,27.032720,27.258259,26.947135,358722.0,-0.008420,0.016220,0.000075


In [5]:

# Feature 1: Daily Return
df['daily_return'] = df['close'].pct_change()

# Feature 2: 7-Day Rolling Volatility
df['rolling_volatility'] = df['daily_return'].rolling(window=7).std()

# Feature 3: Price-to-Volume Ratio
df['price_volume_ratio'] = df['close'] / df['volume']

df.head(10)



C:\Users\kvssr\AppData\Local\Temp\ipykernel_38144\819280260.py:2: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['daily_return'] = df['close'].pct_change()


,open,close,high,low,volume,daily_return,rolling_volatility,price_volume_ratio
2024-01-01,25.045893,25.000000,25.106628,24.871937,264134.0,NaN,NaN,0.000095
2024-01-02,25.353451,25.246297,25.433061,24.885822,175713.0,0.009852,NaN,0.000144
2024-01-03,25.242928,25.190910,25.355092,24.896416,318335.0,-0.002194,NaN,0.000079
2024-01-04,25.434250,25.511774,25.594675,25.358924,175418.0,0.012737,NaN,0.000145
2024-01-05,26.242240,26.267680,26.362557,25.896931,228232.0,0.029630,NaN,0.000115
2024-01-08,26.228890,26.162599,26.359807,25.957902,263038.0,-0.004000,NaN,0.000099
2024-01-09,26.021065,26.057946,26.147253,25.669163,268383.0,-0.004000,NaN,0.000097
2024-01-10,26.882378,26.858537,27.246761,26.530918,393719.0,0.030724,0.015066,0.000068
2024-01-11,27.304503,27.262266,27.362478,27.072712,425996.0,0.015032,0.015162,0.000064
2024-01-12,26.982605,27.032720,27.258259,26.947135,358722.0,-0.008420,0.016220,0.000075
